### Final Project Submission

* Student name:                                 CHRISTINE NKIROTE NTOITI
* Student pace:                                 part time 
* Scheduled project review date/time: 
* Instructor name:                              DANIEL EKALE/SAMUEL KARU
* Blog post URL:


# Movies Analysis


## Project Overview
The project aims to provide actionable insights to guide the newly established movie studio in making data-driven decisions on which films to produce by identifying successful genres, themes, and key factors such as budget, cast, marketing, and critical reception. 
The project's primary audience includes the Head of the Movie Studio, the Creative Development Team, and the Marketing Team. These teams will use these insights to inform long-term planning, content creation, and promotional strategies. 
Key questions to be explored include the relationship between genre and box office revenue, the impact of budgets and reviews on performance, and the influence of actors, directors, and release timing. 
The deliverables will include a detailed analysis report, actionable recommendations, visualizations of trends and patterns, and potentially a predictive model for estimating box office success, all aimed at helping the movie studio produce films that align with market trends and maximize potential returns.





## Business Problem
The newly established movie studio, is keen to determine how to translate historical performance data into strategic decisions about film development, to produce content that is more likely to succeed commercially and meet audience demand. 
Despite being in a highly competitive industry, the movie studio lacks the data-driven insights needed to make informed decisions about which types of films to produce. Without understanding current trends in the movie market, including the genres, themes, budgets, cast, and production values that are most likely to succeed at the box office, the studio risks investing in film projects that may underperform or fail to attract audiences. The company is seeking to leverage data from multiple reputable sources (Box Office Mojo, IMDB, Rotten Tomatoes, The MovieDB, and The Numbers) to uncover actionable insights that will guide the creative, financial, and marketing decisions for the studio.

## Data Understanding
This project aims to analyze the current movie landscape using datasets from sources like [Box Office Mojo](https://github.com/ntoitichristine/movies-phase-2-project-/blob/main/zippedData/bom.movie_gross.csv.gz)
 ,[IMDb]https://github.com/ntoitichristine/movies-phase-2-project-/blob/main/zippedData/im.db.zipB , Rotten Tomatose](https://github.com/ntoitichristine/movies-phase-2-project-/blob/main/zippedData/rt.movie_info.tsv.gzs, The MovieBD](https://github.com/ntoitichristine/movies-phase-2-project-/blob/main/zippedData/tmdb.movies.csv.gzB an [ The Numbesr](https://github.com/ntoitichristine/movies-phase-2-project-/blob/main/zippedData/tn.movie_budgets.csv.gz)  to uncover trends and patterns in the types of films that perform well at the box office. 

In [3]:
#install all libraries to be used
import pandas as pd
import numpy as np
import seaborn as sns
import scipy as sp
import scipy.stats as st
import sqlite3
import zipfile
import os
import statsmodels.api as sm
from statsmodels.stats.power import TTestIndPower, TTestPower
import statsmodels.formula as smf
import matplotlib.pyplot as plt
%matplotlib inline
# Ignore all warnings
import warnings
warnings.filterwarnings('ignore')

### Box Office Movies Data

In [4]:
#file to the bom zip file
bom_path = "./zippedData/bom.movie_gross.csv.gz"
#read the csv file
bom_df = pd.read_csv(bom_path)
bom_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [5]:
bom_df.describe()

,domestic_gross,year
count,3.359000e+03,3387.000000
mean,2.874585e+07,2013.958075
std,6.698250e+07,2.478141
min,1.000000e+02,2010.000000
25%,1.200000e+05,2012.000000
50%,1.400000e+06,2014.000000
75%,2.790000e+07,2016.000000
max,9.367000e+08,2018.000000


In [7]:
#convert 
bom_df.isnull().sum()

title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64

### IMDb Data
This is a SQLite database file containing information about the movies production, detailing movie basics, directors, cast, writers, etc. 

In [9]:
#file to the imdb zip file
imdb_zip_path = "./zippedData/im.db.zip"
#path to extracted imdb file
imdb_path = "./zippedData/im.db"

#extract the SQLite database from the zipfile
with zipfile.ZipFile(imdb_zip_path, 'r') as zip_ref:
    zip_ref.extractall(os.path.dirname(imdb_path))  
    
#create a connection
conn = sqlite3.connect(imdb_path)
#initialize a cursor
cur = conn.cursor()

In [5]:
# show tables in the database

tables = """
            SELECT name 
            AS 'Table Names' 
            FROM sqlite_master 
            WHERE type='table';"""

pd.read_sql(tables, conn)

,Table Names
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


In [11]:
#load the tables into pandas DataFrames
movie_basics = pd.read_sql("SELECT * FROM movie_basics", conn)
directors = pd.read_sql("SELECT * FROM directors", conn)
known_for = pd.read_sql("SELECT * FROM known_for", conn)
movie_akas = pd.read_sql("SELECT * FROM movie_akas", conn)
movie_ratings = pd.read_sql("SELECT * FROM movie_ratings", conn)
persons = pd.read_sql("SELECT * FROM persons", conn)
principals = pd.read_sql("SELECT * FROM principals", conn)
writers = pd.read_sql("SELECT * FROM writers", conn)

# Inspect the data
movie_basics.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


### Rotten Tomatoes: Movie Info Data


In [12]:
#paths to rating files
rt_info_path = "./zippedData/rt.movie_info.tsv.gz"

#read the tsv file
rt_info_df = pd.read_csv(rt_info_path, sep='\t')
rt_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


In [8]:
#convert datatypes
#convert theatre_date column to datetype
rt_info_df['theater_date'] = pd.to_datetime(rt_info_df['theater_date'], errors='coerce')

#convert dvd_date column to datetype
rt_info_df['dvd_date'] = pd.to_datetime(rt_info_df['dvd_date'], errors='coerce')

#convert currency column to numeric
#rt_info_df['currency'] = pd.to_numeric(rt_info_df['currency'], errors='coerce')

#remove 'minutes' from the data and convert column to float
rt_info_df['runtime'] = rt_info_df['runtime'].replace({r'minutes': '', r',': ''}, regex=True)
                                      
#convert runtime to numeric
rt_info_df['runtime'] = pd.to_numeric(rt_info_df['runtime'], errors='coerce')

rt_info_df.head()
#rt_info_df.info()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,1971-10-09,2001-09-25,NaN,NaN,104.0,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,2012-08-17,2013-01-01,$,"600,000",108.0,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,1996-09-13,2000-04-18,NaN,NaN,116.0,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,1994-12-09,1997-08-27,NaN,NaN,128.0,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaT,NaT,NaN,NaN,200.0,NaN


In [9]:
rt_info_df.columns

Index(['id', 'synopsis', 'rating', 'genre', 'director', 'writer',
       'theater_date', 'dvd_date', 'currency', 'box_office', 'runtime',
       'studio'],
      dtype='object')

### Rotten Tomatoes: Reviews Data

In [13]:
#paths to rating files
rt_reviews_path = "./zippedData/rt.reviews.tsv.gz"

#read the tsv file
rt_reviews_df = pd.read_csv(rt_reviews_path, sep='\t', encoding='ISO-8859-1')

rt_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54432 non-null  int64 
 1   review      48869 non-null  object
 2   rating      40915 non-null  object
 3   fresh       54432 non-null  object
 4   critic      51710 non-null  object
 5   top_critic  54432 non-null  int64 
 6   publisher   54123 non-null  object
 7   date        54432 non-null  object
dtypes: int64(2), object(6)
memory usage: 3.3+ MB


In [11]:
#convert top_critic column to numeric
rt_reviews_df['top_critic'] = pd.to_numeric(rt_reviews_df['top_critic'], errors='coerce')

#convert date column to datetype
rt_reviews_df['date'] = pd.to_datetime(rt_reviews_df['date'], errors='coerce')

rt_reviews_df.head()
#rt_reviews_df.info()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,2018-11-10
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,2018-05-23
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,2018-01-04
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,2017-11-16
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,2017-10-12


In [12]:
rt_reviews_df.columns

Index(['id', 'review', 'rating', 'fresh', 'critic', 'top_critic', 'publisher',
       'date'],
      dtype='object')

### TMDb Data

In [14]:
#path to the tmdb file
tmdb_path = "./zippedData/tmdb.movies.csv.gz"

#read the tsv file
tmdb_df = pd.read_csv(tmdb_path, index_col=0)

tmdb_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26517 entries, 0 to 26516
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   genre_ids          26517 non-null  object 
 1   id                 26517 non-null  int64  
 2   original_language  26517 non-null  object 
 3   original_title     26517 non-null  object 
 4   popularity         26517 non-null  float64
 5   release_date       26517 non-null  object 
 6   title              26517 non-null  object 
 7   vote_average       26517 non-null  float64
 8   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 2.0+ MB


In [14]:
#convert release_date column to datetype
tmdb_df['release_date'] = pd.to_datetime(tmdb_df['release_date'], errors='coerce')

tmdb_df.head()
#tmdb_df.info()


,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


### The Numbers Movie_Budgets Data

In [15]:
#create a path to the file
tn_path = "./zippedData/tn.movie_budgets.csv.gz"

#read the csv file
tn_df = pd.read_csv(tn_path)
tn_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [16]:
#convert release_date to datetype
tn_df['release_date'] = pd.to_datetime(tn_df['release_date'], errors='coerce')

#remove the '$' symbol and convert the columns to float
tn_df['production_budget'] = tn_df['production_budget'].replace({'\\$': '', ',': ''}, regex=True)
tn_df['domestic_gross'] = tn_df['domestic_gross'].replace({'\\$': '', ',': ''}, regex=True)
tn_df['worldwide_gross'] = tn_df['worldwide_gross'].replace({'\\$': '', ',': ''}, regex=True)

#convert the columns to numeric
tn_df['production_budget'] = pd.to_numeric(tn_df['production_budget'])
tn_df['domestic_gross'] = pd.to_numeric(tn_df['domestic_gross'])
tn_df['worldwide_gross'] = pd.to_numeric(tn_df['worldwide_gross'])

#rename 'movie' column to 'title'
tn_df = tn_df.rename(columns = {'movie': 'title'})

tn_df.head()
#tn_df.info()

,id,release_date,title,production_budget,domestic_gross,worldwide_gross
0,1,2009-12-18,Avatar,425000000,760507625,2776345279
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
2,3,2019-06-07,Dark Phoenix,350000000,42762350,149762350
3,4,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1403013963
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747


## Data Preparation

### Data Cleaning
For the files, and Database, I normalize column names, and drop unnecessary columns for ease of working with the data.

#### Box Office Movies

In [17]:
#
bom_df['foreign_gross'] = pd.to_numeric(bom_df['foreign_gross'], errors='coerce')

#convert column names to lowercase and strip unnecessary characters and spaces
bom_df.columns = bom_df.columns.str.lower().str.replace(' ', '_')

# drop rows where critical data is missing
bom_df.dropna(subset=['domestic_gross', 'foreign_gross', 'studio'], inplace=True)

# identify missing values
print(bom_df.isnull().sum())

title             0
studio            0
domestic_gross    0
foreign_gross     0
year              0
dtype: int64


#### Rotten Tomatoes movie info and Reviews DataFrames preparation

In [18]:
#convert column names to lowercase and strip unnecessary characters and spaces
rt_info_df.columns = rt_info_df.columns.str.lower().str.replace(' ', '_')

# drop rows where critical data is missing
rt_info_df.dropna(subset=['currency', 'box_office', 'studio', 'director', 'writer', 'theater_date', 'dvd_date'], inplace=True)

# identify missing values
print(rt_info_df.isnull().sum())

id              0
synopsis        0
rating          0
genre           0
director        0
writer          0
theater_date    0
dvd_date        0
currency        0
box_office      0
runtime         1
studio          0
dtype: int64


In [19]:
#convert column names to lowercase and strip unnecessary characters and spaces
rt_reviews_df.columns = rt_reviews_df.columns.str.lower().str.replace(' ', '_')

#drop columns with high number of missing values
#rt_reviews_df.drop(columns = ['rating', 'critic'], inplace=True)

# drop rows where critical data is missing
rt_reviews_df.dropna(subset=['review', 'rating', 'critic', 'publisher'], inplace=True)

# identify missing values
print(rt_reviews_df.isnull().sum())

id            0
review        0
rating        0
fresh         0
critic        0
top_critic    0
publisher     0
date          0
dtype: int64


#### Merging Rotten Tomatoes Movie Info df and Reviews df

In [20]:
# Merge 'rt_info_df' with 'rt_reviews_df' on the 'id' column using a left join
rt_merged_df = pd.merge(rt_info_df, rt_reviews_df, on="id", how="left")

# Rename the columns for clarity
rt_merged_df.rename(columns={'rating_x': 'info_rating', 'rating_y': 'review_rating'}, inplace=True)

# Check the result
print(rt_merged_df.head())


   id                                           synopsis info_rating  \
0   3  New York City, not-too-distant-future: Eric Pa...           R   
1   3  New York City, not-too-distant-future: Eric Pa...           R   
2   3  New York City, not-too-distant-future: Eric Pa...           R   
3   3  New York City, not-too-distant-future: Eric Pa...           R   
4   3  New York City, not-too-distant-future: Eric Pa...           R   

                               genre          director  \
0  Drama|Science Fiction and Fantasy  David Cronenberg   
1  Drama|Science Fiction and Fantasy  David Cronenberg   
2  Drama|Science Fiction and Fantasy  David Cronenberg   
3  Drama|Science Fiction and Fantasy  David Cronenberg   
4  Drama|Science Fiction and Fantasy  David Cronenberg   

                         writer theater_date   dvd_date currency box_office  \
0  David Cronenberg|Don DeLillo   2012-08-17 2013-01-01        $    600,000   
1  David Cronenberg|Don DeLillo   2012-08-17 2013-01-01     

In [21]:
# Split the 'genre' column and keep only the first genre
rt_merged_df['primary_genre'] = rt_merged_df['genre'].str.split('|').str[0]

# Split the 'writer' column and keep only the first writer
rt_merged_df['primary_writer'] = rt_merged_df['writer'].str.split('|').str[0]

# Drop 'genre', 'writer', 'id', 'synopsis', 'review' columns
rt_merged_df.drop(columns=['genre', 'writer', 'id', 'synopsis', 'review'], inplace=True)

# Remove comma from the 'box_office' column and convert to integer
rt_merged_df['box_office'] = rt_merged_df['box_office'].str.replace(',', ' ', regex=False)

In [22]:
print(rt_merged_df.head())

  info_rating          director theater_date   dvd_date currency box_office  \
0           R  David Cronenberg   2012-08-17 2013-01-01        $    600 000   
1           R  David Cronenberg   2012-08-17 2013-01-01        $    600 000   
2           R  David Cronenberg   2012-08-17 2013-01-01        $    600 000   
3           R  David Cronenberg   2012-08-17 2013-01-01        $    600 000   
4           R  David Cronenberg   2012-08-17 2013-01-01        $    600 000   

   runtime             studio review_rating   fresh          critic  \
0    108.0  Entertainment One           3/5   fresh      PJ Nabarro   
1    108.0  Entertainment One             C  rotten  Eric D. Snider   
2    108.0  Entertainment One           2/5  rotten    Matt Kelemen   
3    108.0  Entertainment One            B-   fresh    Emanuel Levy   
4    108.0  Entertainment One           2/4  rotten  Christian Toto   

   top_critic           publisher       date primary_genre    primary_writer  
0         0.0     P

#### TMDB df Preparation

In [23]:
#convert column names to lowercase and strip unnecessary characters and spaces
tmdb_df.columns = tmdb_df.columns.str.lower().str.replace(' ', '_')

# identify missing values
print(tmdb_df.isnull().sum())

#drop unncessary columns
#tmdb_df.drop(columns = ['original_language', 'original_title'], inplace=True)

genre_ids            0
id                   0
original_language    0
original_title       0
popularity           0
release_date         0
title                0
vote_average         0
vote_count           0
dtype: int64


In [24]:
#convert column names to lowercase and strip unnecessary characters and spaces
tn_df.columns = tn_df.columns.str.lower().str.replace(' ', '_')

# identify missing values
print(tn_df.isnull().sum())


id                   0
release_date         0
title                0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64


#### IMDb Data Preparation

In [25]:
# identify missing values
print(movie_basics.isnull().sum())
print(directors.isnull().sum())
print(movie_ratings.isnull().sum())
print(known_for.isnull().sum())
print(movie_akas.isnull().sum())
print(movie_ratings.isnull().sum())
print(persons.isnull().sum())
print(principals.isnull().sum())
print(writers.isnull().sum())

movie_id               0
primary_title          0
original_title        21
start_year             0
runtime_minutes    31739
genres              5408
dtype: int64
movie_id     0
person_id    0
dtype: int64
movie_id         0
averagerating    0
numvotes         0
dtype: int64
person_id    0
movie_id     0
dtype: int64
movie_id                  0
ordering                  0
title                     0
region                53293
language             289988
types                163256
attributes           316778
is_original_title        25
dtype: int64
movie_id         0
averagerating    0
numvotes         0
dtype: int64
person_id                  0
primary_name               0
birth_year            523912
death_year            599865
primary_profession     51340
dtype: int64
movie_id           0
ordering           0
person_id          0
category           0
job           850502
characters    634826
dtype: int64
movie_id     0
person_id    0
dtype: int64


In [26]:
# Define a function to Drop columns with more than 5% missing data
def drop_columns(df, threshold=5):
    # Calculate the percentage of missing data per column
    missing_percentage = df.isnull().sum() / len(df) * 100
    
    # Identify columns with missing data greater than the threshold
    columns_to_drop = missing_percentage[missing_percentage > threshold].index
    
    # Drop those columns
    df.drop(columns=columns_to_drop, inplace=True)
     # Print the columns that were dropped
    if len(columns_to_drop) > 0:
        print(f"Dropped columns with more than {threshold}% missing data: {', '.join(columns_to_drop)}")
    else:
        print(f"No columns dropped from the DataFrame (threshold = {threshold}%)")

# Apply this function to each DataFrame
drop_columns(movie_basics)
drop_columns(directors)
drop_columns(movie_ratings)
drop_columns(known_for)
drop_columns(movie_akas)
drop_columns(persons)
drop_columns(principals)
drop_columns(writers)

Dropped columns with more than 5% missing data: runtime_minutes
No columns dropped from the DataFrame (threshold = 5%)
No columns dropped from the DataFrame (threshold = 5%)
No columns dropped from the DataFrame (threshold = 5%)
Dropped columns with more than 5% missing data: region, language, types, attributes
Dropped columns with more than 5% missing data: birth_year, death_year, primary_profession
Dropped columns with more than 5% missing data: job, characters
No columns dropped from the DataFrame (threshold = 5%)


In [27]:
# Check values before dropping duplicates
print(f"movie_basics shape: {movie_basics.shape}")
print(f"directors shape: {directors.shape}")
print(f"movie_ratings shape: {movie_ratings.shape}")
print(f"known_for shape: {known_for.shape}")
print(f"movie_akas shape: {movie_akas.shape}")
print(f"persons shape: {persons.shape}")
print(f"principals shape: {principals.shape}")
print(f"writers shape before dropping duplicates: {writers.shape}")

movie_basics shape: (146144, 5)
directors shape: (291174, 2)
movie_ratings shape: (73856, 3)
known_for shape: (1638260, 2)
movie_akas shape: (331703, 4)
persons shape: (606648, 2)
principals shape: (1028186, 4)
writers shape before dropping duplicates: (255873, 2)


In [28]:
# Drop duplicates
def drop_duplicates(df, subset=None, keep='first', inplace=True):
    # Drop duplicates based on the subset of columns (if provided)
    if subset:
        df.drop_duplicates(subset=subset, keep=keep, inplace=inplace)
    else:
        df.drop_duplicates(keep=keep, inplace=inplace)

    return df

#Apply the function with appropriate subset of columns for each DataFrame
drop_duplicates(movie_basics, subset=['movie_id'], keep='first')
drop_duplicates(directors, subset=['person_id'], keep='first')
drop_duplicates(movie_ratings, subset=['movie_id'], keep='first')
drop_duplicates(known_for, subset=['movie_id', 'person_id'], keep='first')
drop_duplicates(movie_akas, subset=['movie_id'], keep='first') 
drop_duplicates(persons, subset=['person_id'], keep='first')
drop_duplicates(principals, subset=['movie_id', 'person_id'], keep='first')
drop_duplicates(writers, subset=['person_id', 'movie_id'], keep='first')

# Verify the results after dropping duplicates
print(f"clean movie_basics shape: {movie_basics.shape}")
print(f"clean directors shape: {directors.shape}")
print(f"clean movie_ratings shape: {movie_ratings.shape}")
print(f"clean known_for shape: {known_for.shape}")
print(f"clean movie_akas shape: {movie_akas.shape}")
print(f"clean persons shape: {persons.shape}")
print(f"clean principals shape: {principals.shape}")
print(f"clean writers shape: {writers.shape}")

clean movie_basics shape: (146144, 5)
clean directors shape: (109253, 2)
clean movie_ratings shape: (73856, 3)
clean known_for shape: (1638260, 2)
clean movie_akas shape: (122302, 4)
clean persons shape: (606648, 2)
clean principals shape: (1028148, 4)
clean writers shape: (178352, 2)


### Merging IMDb DataFrames

In [29]:
# Merge `movie_basics` with `movie_ratings` on `movie_id`
imdb_merged_df = pd.merge(movie_basics, movie_ratings, on="movie_id", how="left")

# Merge with `movie_akas` on `movie_id`
imdb_merged_df = pd.merge(imdb_merged_df, movie_akas, on="movie_id", how="left")

# Merge with `directors` on `movie_id` (many-to-many relation)
imdb_merged_df = pd.merge(imdb_merged_df, directors, on="movie_id", how="left")

# Merge with `principals` on `movie_id` and `person_id` (many-to-many relation)
imdb_merged_df = pd.merge(imdb_merged_df, principals, on=["movie_id", "person_id"], how="left")

# Rename person_id columns after merging to avoid confusion (if necessary)
imdb_merged_df.rename(columns={'person_id_x': 'merged_person_id', 'person_id_y': 'original_person_id'}, inplace=True)

# Merge with `persons` on `person_id`
imdb_merged_df = pd.merge(imdb_merged_df, persons, on="person_id", how="left", suffixes=('_merged', '_person'))

# Merge with `known_for` on `movie_id`
imdb_merged_df = pd.merge(imdb_merged_df, known_for, on="movie_id", how="left")

# Merge with `writers` on `movie_id` and `person_id` (many-to-many relation)
imdb_merged_df = pd.merge(imdb_merged_df, writers, left_on=["movie_id", "person_id_x"], right_on=["movie_id", "person_id"], how="left", suffixes=('_merged', '_writer'))

# Check the result
print(imdb_merged_df.head())

    movie_id                    primary_title              original_title  \
0  tt0063540                        Sunghursh                   Sunghursh   
1  tt0063540                        Sunghursh                   Sunghursh   
2  tt0066787  One Day Before the Rainy Season             Ashad Ka Ek Din   
3  tt0069049       The Other Side of the Wind  The Other Side of the Wind   
4  tt0069049       The Other Side of the Wind  The Other Side of the Wind   

   start_year              genres  averagerating  numvotes  ordering_x  \
0        2013  Action,Crime,Drama            7.0      77.0         1.0   
1        2013  Action,Crime,Drama            7.0      77.0         1.0   
2        2019     Biography,Drama            7.2      43.0         1.0   
3        2018               Drama            6.9    4517.0        10.0   
4        2018               Drama            6.9    4517.0        10.0   

                             title  is_original_title person_id_x  ordering_y  \
0          

In [30]:
# Split the 'genres' column and keep only the first genre 
imdb_merged_df['primary_genre'] = imdb_merged_df['genres'].str.split(',').str[0] 

#drop the original 'genres' column 
imdb_merged_df.drop(columns=['genres'], inplace=True)

# Check the result
print(imdb_merged_df[['primary_genre']].head())


  primary_genre
0        Action
1        Action
2     Biography
3         Drama
4         Drama


## Exploratory Data Analysis